In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import keras_nlp

from keras.backend import clear_session
from keras.models import Model
from keras.layers import TextVectorization, Input, Dense, Embedding, Dropout, GlobalAveragePooling1D, LSTM, Bidirectional

from sklearn.model_selection import train_test_split

2024-05-22 07:02:47.393481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 07:02:47.393586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 07:02:47.506509: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class Config():
    def __init__(self):
        self.url = "https://s3.eu-west-3.amazonaws.com/profession.ai/datasets/Filter_Toxic_Comments_dataset.csv"
        self.max_tokens = 20000
        self.output_sequence_length = 200
        self.embedding_dim = 128
        self.batch_size = 32
        self.epochs = 10
        self.train_split = 0.7
        self.test_split = 0.15
        self.val_split = 0.15
        self.random_state = 42
        self.features = 'comment_text'
        self.labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        self.new_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', "clean"]
        self.label_mapping = {label: i for i, label in enumerate(self.labels)}
        self.new_label_mapping = {label: i for i, label in enumerate(self.new_labels)}

    def split_dataset(self, df):
      x = df[self.features]
      y = df[self.labels]
      xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=self.test_split, random_state=self.random_state)
      xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=self.val_split, random_state=self.random_state)
      
      return (xtrain, ytrain), (xtest, ytest), (xval, yval)
    
    def early_stopping(self):
      early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
            verbose=0,
            mode="auto",
            baseline=None,
            restore_best_weights=True,
            start_from_epoch=0
        )
      return early_stopping
    
    def model_checkpoint(self, filepath):
      
      model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="auto",
        save_freq="epoch",
        initial_value_threshold=None
      )
      
      return model_checkpoint

config = Config()

In [4]:
file = tf.keras.utils.get_file("Filter_Toxic_Comments_dataset.csv", config.url)
df = pd.read_csv(file)
df.head()

66034407/66034407 ━━━━━━━━━━━━━━━━━━━━ 20s 0us/step


,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum_injurious
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


In [5]:
(xtrain, ytrain), (xtest, ytest), (xval, yval) = config.split_dataset(df)

In [6]:
x = df[config.features]
y = df[config.labels]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=config.test_split, random_state=config.random_state)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=config.val_split, random_state=config.random_state)


In [7]:
print(
  f"xtrain shape: {xtrain.shape}\n",
  f"ytrain shape: {ytrain.shape}\n",
  f"xtest shape: {xtest.shape}\n",
  f"ytest shape: {ytest.shape}\n",
  f"xval shape: {xval.shape}\n",
  f"yval shape: {yval.shape}\n"
  )

xtrain shape: (115289,)
 ytrain shape: (115289, 6)
 xtest shape: (23936,)
 ytest shape: (23936, 6)
 xval shape: (20346,)
 yval shape: (20346, 6)



In [8]:
train_ds = tf.data.Dataset.from_tensor_slices(xtrain).batch(config.batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(xtest).batch(config.batch_size)
val_ds = tf.data.Dataset.from_tensor_slices(xval).batch(config.batch_size)

In [9]:
text_vectorizer = TextVectorization(
    max_tokens=config.max_tokens,
    output_sequence_length=config.output_sequence_length,
    output_mode='int',
    pad_to_max_tokens=True,
    standardize='lower_and_strip_punctuation',
    split='whitespace'
)

text_vectorizer.adapt(xtrain)
#text_vectorizer.get_vocabulary()[:100]

In [10]:
def get_lstm_model():
  clear_session()
  # Input for variable-length sequences of integers
  inputs = Input(shape=(None,), dtype=tf.string)
  # Embed each integer in a 128-dimensional vector
  text_vec = text_vectorizer(inputs)
  x = Embedding(config.max_tokens + 1, 128)(text_vec)
  # Add 2 bidirectional LSTMs
  x = Bidirectional(LSTM(64, return_sequences=True))(x)
  x = Bidirectional(LSTM(64))(x)
  # Add a classifier
  outputs = Dense(len(config.labels), activation="softmax")(x)
  model = Model(inputs, outputs)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  return model

lstm_model = get_lstm_model()
lstm_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 200)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 200, 128)       │     2,560,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200, 128)       │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,758,534 (10.52 MB)

 Trainable params: 2,758,534 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
early_stopping = config.early_stopping()
model_checkpoint = config.model_checkpoint(filepath="lstm_model.keras")

In [12]:
history = lstm_model.fit(
    xtrain, ytrain,
    validation_data=(xval, yval),
    epochs=10,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

Epoch 1/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 165s 44ms/step - accuracy: 0.9818 - loss: 0.3410 - val_accuracy: 0.9478 - val_loss: 0.3308
Epoch 2/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 159s 44ms/step - accuracy: 0.9582 - loss: 0.3303 - val_accuracy: 0.9918 - val_loss: 0.3185
Epoch 3/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 159s 44ms/step - accuracy: 0.9869 - loss: 0.3326 - val_accuracy: 0.9584 - val_loss: 0.3187
Epoch 4/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 158s 44ms/step - accuracy: 0.9877 - loss: 0.3298 - val_accuracy: 0.9918 - val_loss: 0.3155
Epoch 5/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 159s 44ms/step - accuracy: 0.9927 - loss: 0.3295 - val_accuracy: 0.9938 - val_loss: 0.3141
Epoch 6/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 201s 44ms/step - accuracy: 0.9929 - loss: 0.3285 - val_accuracy: 0.9917 - val_loss: 0.3187
Epoch 7/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 158s 44ms/step - accuracy: 0.9901 - loss: 0.3308 - val_accuracy: 0.9907 - val_loss: 0.3154
Epoch 8/10
3603/3603 ━━━━━━━━━━━━━━━━━━━━ 202s 44ms/step - accuracy: 

In [13]:
history_df = pd.DataFrame(history.history)
history_df.to_excel("history.xlsx")